In [1]:
# # Use this the first time setting up the conda environment
#!pip install mysql-connector-python
#!pip install pymysql
#!pip install ipython-sql pymysql
#!pip install cryptography

import pymysql
import pandas as pd
import hashlib
import csv

# Base functions referenced in future cells
def hash_password(password): # kayla update this with your preferred hash
    return hashlib.sha256(password.encode()).hexdigest()

In [2]:
### Fill in your user data
user_info = {
    'username': 'root',
    'password': 'P3nnyTh3D0g'
}

user_info = {
    'username': 'rysch01',
    'password': 'SQLP4ssword1!'
}

In [3]:
# Run this cell and it will either connect to your existing database or create it from the sample data
# Connect to the MySQL serve
db = pymysql.connect(
    host='localhost',    
    user=user_info['username'],
    password=user_info['password'],
)

cursor = db.cursor()

try: # try to use database; if so, it's already loaded
    cursor.execute("USE cafeteria")
except: # if not, need to load it in next
    cursor.execute('CREATE DATABASE cafeteria;')
    cursor.execute("USE cafeteria")
    ### Define the students table
    declare_students='''
    CREATE TABLE Students (
        pid INT PRIMARY KEY,
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255) NOT NULL,
        meal_balance DECIMAL(8, 2) DEFAULT 0.00,
        to_go_boxes_remaining INT DEFAULT 2,
        password_hash CHAR(64) NOT NULL,
        role CHAR(8) DEFAULT "student"
    );
    '''

    ### Define the menu table
    declare_menu='''
    CREATE TABLE Menu (
        item_id INT PRIMARY KEY AUTO_INCREMENT,
        item_name VARCHAR(255) NOT NULL,
        calories_per_serving INT NOT NULL,
        meal_type CHAR(64) NOT NULL,
        available_date DATE NOT NULL,
        price DECIMAL(8, 2) NOT NULL
    );
    '''

    ### Define the transactions table
    declare_transactions='''
    CREATE TABLE Transactions (
        transaction_id INT PRIMARY KEY AUTO_INCREMENT,
        pid INT NOT NULL,
        item_id INT NOT NULL,
        transaction_type VARCHAR(50) NOT NULL,
        transaction_date DATE NOT NULL,
        FOREIGN KEY (pid) REFERENCES Students(pid),
        FOREIGN KEY (item_id) REFERENCES Menu(item_id)
    );
    '''

    # execute and commit changes
    cursor.execute(declare_students)
    cursor.execute(declare_menu)
    cursor.execute(declare_transactions)
    db.commit()

    # load from csv like in HW2

    # Do so for each of the tables
    with open('data/students.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            r[-1]=hash_password(r[-1]) # hash password to store
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Students ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s, %s)', r)

    
    with open('data/menu.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Menu ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s)', r)

    with open('data/transactions.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Transactions ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s)', r)
    # Commit changes
    db.commit()

In [6]:
# # ## Insert Query Example
# new_item_details = (5, 'Omelette', 600, 'breakfast', '2024-11-12', 8.00)
# cursor.execute('INSERT INTO Menu (item_id, item_name, calories_per_serving, meal_type, available_date, price) VALUES (%s, %s, %s, %s, %s, %s);', new_item_details)
# db.commit()

# # ## Select Query Example
# cursor.execute('SELECT * FROM Menu')
# result = cursor.fetchall()
# print(result)

In [4]:
### Sign-In Page Backend Code + Queries

test_case='admin'

if test_case=='student':
    entered_pid='730330697'
    entered_password='ramses123'
elif test_case=='admin':
    entered_pid='987654321'
    entered_password='flav0rTown!!!'
elif test_case=='wrong_password':
    entered_pid='987654321'
    entered_password='wrong_pass'
elif test_case=='nonexistant_user':
    entered_pid='222222222'
    entered_password='wrongPass'

cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
result = cursor.fetchall()

entered_password_hash=hash_password(entered_password)
try:
    if (result[0][5]==entered_password_hash):
        print('Sign-In Success')
        current_user = {
            'pid': result[0][0],
            'first_name': result[0][1],
            'last_name': result[0][2],
            'balance': result[0][3],
            'to_go_remaining': result[0][4],
            'password_hash': result[0][5],
            'role': result[0][6],
        }
        print(current_user)

        if current_user['role']=='admin':
            print('direct to admin page')
        else:
            print('direct to student')
    else:
        print('incorrect password code')
except:
    print('pid not found')


Sign-In Success
{'pid': 987654321, 'first_name': 'Guy', 'last_name': 'Feeyarey', 'balance': Decimal('0.00'), 'to_go_remaining': 0, 'password_hash': '782af4ba40f9fbae4c081fe35a3d1220d193684e05d8776c71bbafe162d86c44', 'role': 'admin'}
direct to admin page


In [5]:
### Registration Code: Backend + Queries

test_case='correct_input'
# 730473910,Kayla,Casey,testPassword
if test_case=='correct_input':
    entered_pid='756589321'
    entered_first='john'
    entered_last='doe'
    entered_password1='passieword'
    entered_password2='passieword'
elif test_case=='pid_in_use':
    entered_pid='730330697'
    entered_first='jerry'
    entered_last='doubles'
    entered_password1='passieword'
    entered_password2='passieword'
elif test_case=='password_mismatch':
    entered_pid='756589322'
    entered_first='jack'
    entered_last='downs'
    entered_password1='passieword1'
    entered_password2='passieword'
elif test_case=='missing_field_pid':
    entered_pid=''
    entered_first='jenny'
    entered_last='dawkins'
    entered_password1='passieword2'
    entered_password2='passieword2'
elif test_case=='missing_field_first_name':
    entered_pid='756589323'
    entered_first=''
    entered_last='deere'
    entered_password1='passieword3'
    entered_password2='passieword3'
elif test_case=='missing_field_last_name':
    entered_pid='756589324'
    entered_first='janice'
    entered_last=''
    entered_password1='passieword4'
    entered_password2='passieword4'
elif test_case=='missing_field_password1':
    entered_pid='756589325'
    entered_first='james'
    entered_last='duley'
    entered_password1=''
    entered_password2='passieword5'
elif test_case=='missing_field_password2':
    entered_pid='756589326'
    entered_first='javier'
    entered_last='dalton'
    entered_password1='passieword6'
    entered_password2=''


if entered_pid=='' or not entered_pid.isdigit() or len(entered_pid) != 9: # (check 9 digit num)
    print('Enter valid PID')
elif entered_first=='' or not entered_first.isalpha(): # check not empty, only letters (no num, whitespace)
    print('Enter valid First Name')
elif entered_last=='' or not entered_last.isalpha(): # check not empty, only letters (no num, whitespace)
    print('Enter valid Last Name')
elif entered_password1=='' or len(entered_password1) > 24 or len(entered_password1) < 8:  # checks empty, add other check for validity
    print('Enter valid password; must be between 8 and 24 characters')
elif entered_password2 != entered_password1: # triggers if first valid but second doesn't match
    print('Passwords do Not Match')
else: # means all valid

    # Check if PID in use
    cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
    result = cursor.fetchall()
    try:
        result[0][0] # if returns a result, it's being used
        print('PID Already in Use')
    except: # may be a bad idea as any error will allow it to continue, but I think the only error for this would be an index error meaning that there's no PID in use
        new_user_details = (entered_pid, entered_first.title(), entered_last.title(), hash_password(entered_password1))
        cursor.execute('INSERT INTO Students (pid, first_name, last_name, password_hash) VALUES (%s, %s, %s, %s);', new_user_details)
        db.commit()

        cursor.execute('SELECT * FROM Students S WHERE S.pid=%s', entered_pid)
        result = cursor.fetchall()

        print('Sign-Up Success')
        current_user = {
            'pid': result[0][0],
            'first_name': result[0][1],
            'last_name': result[0][2],
            'balance': result[0][3],
            'to_go_remaining': result[0][4],
            'password_hash': result[0][5],
            'role': result[0][6],
        }
        print(current_user)

PID Already in Use


In [6]:
### Landing Page
# based on current user from sign-in cell; 
today_date='2024-11-13'

### Header 
# Common Info
# print(f'Name: {current_user["first_name"]} {current_user["last_name"]}')
print(f'Name: {current_user["first_name"]} {current_user["last_name"]}')
print(f'PID: {current_user["pid"]}')
print(f'{current_user["role"].title()} View')

# Right Header
if current_user["role"]=='student':
    print(f'Account Balance: ${current_user["balance"]}')
    print(f'To-Go Boxes Remaining: {current_user["to_go_remaining"]}/2')
if current_user["role"]=='admin':
    print('Button for New Order')
    print('Button for Student Management')

# Transaction History
if current_user["role"]=='student':
    print("Your Transaction History")
    cursor.execute('SELECT * FROM Transactions T WHERE T.pid=%s', current_user["pid"])
    result = cursor.fetchall()
    print(result)
elif current_user["role"]=='admin':
    print("Transaction History")
    cursor.execute('SELECT * FROM Transactions T')
    result = cursor.fetchall()
    print(result)

# Today's Menu
print("Today's Menu")
cursor.execute('SELECT * FROM Menu M where available_date=%s', today_date)
result = cursor.fetchall()
print(result)




Name: Guy Feeyarey
PID: 987654321
Admin View
Button for New Order
Button for Student Management
Transaction History
((1, 730473910, 1, 'dine-in', datetime.date(2024, 11, 10)), (2, 730560194, 2, 'dine-in', datetime.date(2024, 11, 11)), (3, 730330697, 3, 'dine-in', datetime.date(2024, 11, 9)), (4, 730578592, 4, 'to-go', datetime.date(2024, 11, 10)), (5, 730330697, 2, 'to-go', datetime.date(2024, 11, 10)))
Today's Menu
((4, 'Vegan Burrito Bowl', 500, 'lunch', datetime.date(2024, 11, 13), Decimal('9.00')),)


In [7]:
import hashlib
import pandas as pd

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

students_df = pd.read_csv('data/students.csv')
students_df['password_hash'] = students_df['password_hash'].apply(hash_password)
students_df.to_csv('data/students_hashed.csv', index=False)

In [8]:
# CREATE DATABASE IF NOT EXISTS cafeteria;

# USE cafeteria;

# CREATE TABLE Students (
#     pid INT PRIMARY KEY,
#     first_name VARCHAR(50),
#     last_name VARCHAR(50),
#     password_hash VARCHAR(255),
#     role ENUM('student', 'admin') DEFAULT 'student',
#     meal_balance DECIMAL(10, 2) DEFAULT 0.0,
#     to_go_boxes INT DEFAULT 0,
#     last_visit_date DATE
# );

# CREATE TABLE Menu (
#     item_id INT PRIMARY KEY AUTO_INCREMENT,
#     item_name VARCHAR(100),
#     calories_per_serving INT,
#     meal_type ENUM('breakfast', 'lunch', 'dinner'),
#     available_date DATE,
#     quantity INT,
#     price DECIMAL(10, 2),
#     prep_time INT
# );

# CREATE TABLE Transactions (
#     transaction_id INT PRIMARY KEY AUTO_INCREMENT,
#     student_id INT,
#     item_id INT,
#     transaction_type ENUM('meal swipe', 'to-go box checkout'),
#     transaction_date DATE,
#     amount DECIMAL(10, 2),
#     FOREIGN KEY (student_id) REFERENCES Students(pid),
#     FOREIGN KEY (item_id) REFERENCES Menu(item_id)
# );

# CREATE TABLE DiningHistory (
#     history_id INT PRIMARY KEY AUTO_INCREMENT,
#     student_id INT,
#     transaction_date DATE,
#     transaction_type ENUM('meal swipe', 'to-go box checkout'),
#     item_id INT,
#     meal_type ENUM('breakfast', 'lunch', 'dinner'),
#     FOREIGN KEY (student_id) REFERENCES Students(pid),
#     FOREIGN KEY (item_id) REFERENCES Menu(item_id)
# );


In [2]:
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QPushButton, QLineEdit, QTextEdit, QTableWidget, QTableWidgetItem,
    QStackedWidget, QHBoxLayout
)
from PyQt5.QtCore import Qt
import pymysql
import sys
import hashlib



class LoginPage(QWidget):
    def __init__(self, switch_to_admin, switch_to_student):
        super().__init__()
        self.switch_to_admin = switch_to_admin
        self.switch_to_student = switch_to_student
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.title_label = QLabel('Login Page', self)
        self.title_label.setAlignment(Qt.AlignCenter)
        self.title_label.setStyleSheet('font-size: 20px; font-weight: bold;')
        layout.addWidget(self.title_label)

        self.pid_input = QLineEdit(self)
        self.pid_input.setPlaceholderText('Enter PID')
        layout.addWidget(self.pid_input)

        self.password_input = QLineEdit(self)
        self.password_input.setPlaceholderText('Enter Password')
        self.password_input.setEchoMode(QLineEdit.Password)
        layout.addWidget(self.password_input)

        self.login_button = QPushButton('Login', self)
        self.login_button.clicked.connect(self.handle_login)
        layout.addWidget(self.login_button)

        self.error_display = QTextEdit(self)
        self.error_display.setReadOnly(True)
        self.error_display.setStyleSheet('color: red;')
        layout.addWidget(self.error_display)

        self.setLayout(layout)

    def handle_login(self):
        pid = self.pid_input.text()
        password = self.password_input.text()
        self.error_display.clear()

        try:
            connection = pymysql.connect(
                host='localhost',
                user='root',
                password='P3nnyTh3D0g',
                database='cafeteria',
            )

            connection = pymysql.connect(
                host='localhost',
                user='rysch01',
                password='SQLP4ssword1!',
                database='cafeteria',
            )
            cursor = connection.cursor()
            cursor.execute("SELECT role, password_hash FROM Students WHERE pid = %s", (pid,))
            result = cursor.fetchone()

            if result:
                role, password_hash = result
                if password_hash == password:  # Simplified for now; replace with hashed check
                    if role == 'admin':
                        self.switch_to_admin()
                    else:
                        self.switch_to_student(pid)
                else:
                    self.error_display.setText("Error: Incorrect password.")
            else:
                self.error_display.setText("Error: PID not found.")

            cursor.close()
            connection.close()
        except Exception as e:
            self.error_display.setText(f"Error: {e}")
    def hash_password(self, password):
        return hashlib.sha256(password.encode()).hexdigest()

    def handle_login(self):
        pid = self.pid_input.text()
        password = self.password_input.text()
        self.error_display.clear()

        try:
            connection = pymysql.connect(
                host='localhost',
                user='root',
                password='P3nnyTh3D0g',
                database='cafeteria',
            )
            cursor = connection.cursor()
            cursor.execute("SELECT role, password_hash FROM Students WHERE pid = %s", (pid,))
            result = cursor.fetchone()

            if result:
                role, password_hash = result
                if password_hash == self.hash_password(password):  # Secure hashed comparison
                    if role == 'admin':
                        self.switch_to_admin()
                    else:
                        self.switch_to_student(pid)
                else:
                    self.error_display.setText("Error: Incorrect password.")
            else:
                self.error_display.setText("Error: PID not found.")

            cursor.close()
            connection.close()
        except Exception as e:
            self.error_display.setText(f"Error: {e}")


class AdminView(QWidget):
    def __init__(self, switch_to_login, backend):
        super().__init__()
        self.switch_to_login = switch_to_login
        self.backend = backend
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.title_label = QLabel('Admin Dashboard', self)
        self.title_label.setAlignment(Qt.AlignCenter)
        self.title_label.setStyleSheet('font-size: 20px; font-weight: bold;')
        layout.addWidget(self.title_label)

        # Buttons for admin functionalities
        manage_students_button = QPushButton("Manage Students")
        manage_students_button.clicked.connect(self.show_manage_students)
        layout.addWidget(manage_students_button)

        manage_menu_button = QPushButton("Manage Menu")
        manage_menu_button.clicked.connect(self.show_manage_menu)
        layout.addWidget(manage_menu_button)

        view_transactions_button = QPushButton("View Transactions")
        view_transactions_button.clicked.connect(self.show_view_transactions)
        layout.addWidget(view_transactions_button)

        logout_button = QPushButton("Logout")
        logout_button.clicked.connect(self.switch_to_login)
        layout.addWidget(logout_button)

        self.setLayout(layout)

    def show_manage_students(self):
        self.clear_screen()
        layout = QVBoxLayout()

        title = QLabel("Manage Students")
        title.setStyleSheet("font-size: 20px; font-weight: bold; margin-bottom: 15px;")
        layout.addWidget(title)

        # Example: Display student details in a table
        student_table = QTableWidget()
        student_data = self.backend.get_all_students()
        student_table.setRowCount(len(student_data))
        student_table.setColumnCount(4)
        student_table.setHorizontalHeaderLabels(["PID", "Name", "Balance", "To-Go Boxes"])

        for row, student in enumerate(student_data):
            for col, value in enumerate(student):
                student_table.setItem(row, col, QTableWidgetItem(str(value)))

        layout.addWidget(student_table)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.initUI)
        layout.addWidget(back_button)

        self.setLayout(layout)

    def show_manage_menu(self):
        self.clear_screen()
        layout = QVBoxLayout()

        title = QLabel("Manage Menu")
        title.setStyleSheet("font-size: 20px; font-weight: bold; margin-bottom: 15px;")
        layout.addWidget(title)

        # Example: Display menu items in a table
        menu_table = QTableWidget()
        menu_data = self.backend.get_all_menu_items()
        menu_table.setRowCount(len(menu_data))
        menu_table.setColumnCount(5)
        menu_table.setHorizontalHeaderLabels(["Item ID", "Name", "Quantity", "Price", "Meal Type"])

        for row, item in enumerate(menu_data):
            for col, value in enumerate(item):
                menu_table.setItem(row, col, QTableWidgetItem(str(value)))

        layout.addWidget(menu_table)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.initUI)
        layout.addWidget(back_button)

        self.setLayout(layout)

    def show_view_transactions(self):
        self.clear_screen()
        layout = QVBoxLayout()

        title = QLabel("View Transactions")
        title.setStyleSheet("font-size: 20px; font-weight: bold; margin-bottom: 15px;")
        layout.addWidget(title)

        # Example: Display transactions in a table
        transaction_table = QTableWidget()
        transaction_data = self.backend.get_all_transactions()
        transaction_table.setRowCount(len(transaction_data))
        transaction_table.setColumnCount(5)
        transaction_table.setHorizontalHeaderLabels(
            ["Transaction ID", "Student ID", "Item ID", "Type", "Amount"]
        )

        for row, transaction in enumerate(transaction_data):
            for col, value in enumerate(transaction):
                transaction_table.setItem(row, col, QTableWidgetItem(str(value)))

        layout.addWidget(transaction_table)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.initUI)
        layout.addWidget(back_button)

        self.setLayout(layout)

    def clear_screen(self):
        # Clear the current screen to show new content
        for i in reversed(range(self.layout().count())):
            widget = self.layout().itemAt(i).widget()
            if widget is not None:
                widget.deleteLater()


class StudentView(QWidget):
    def __init__(self, pid, switch_to_login, backend):
        super().__init__()
        self.pid = pid
        self.switch_to_login = switch_to_login
        self.backend = backend
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        self.title_label = QLabel('Student View', self)
        self.title_label.setAlignment(Qt.AlignCenter)
        self.title_label.setStyleSheet('font-size: 20px; font-weight: bold;')
        layout.addWidget(self.title_label)

        # Display student details
        student_name = self.backend.get_student_name(self.pid)
        account_balance = self.backend.get_account_balance(self.pid)
        togo_boxes = self.backend.get_togo_boxes(self.pid)

        layout.addWidget(QLabel(f"Name: {student_name}"))
        layout.addWidget(QLabel(f"Account Balance: ${account_balance:.2f}"))
        layout.addWidget(QLabel(f"To-Go Boxes Available: {togo_boxes}"))

        # Add more student functionalities (e.g., transaction history, menu view)

        self.logout_button = QPushButton('Logout', self)
        self.logout_button.clicked.connect(self.switch_to_login)
        layout.addWidget(self.logout_button)

        self.setLayout(layout)

    def show_manage_students_screen(self):
        self.clear_screen()
        layout = QVBoxLayout()

        title = QLabel("Manage Students")
        title.setStyleSheet("font-size: 20px; font-weight: bold; margin-bottom: 15px;")
        layout.addWidget(title)

        # Add table or form to edit student details
        # Example: Show list of students in a table and allow admin to edit or delete records
        # (Use QTableView or a similar widget)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.show_admin_screen)
        layout.addWidget(back_button)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)



class MainWindow(QWidget):
    def __init__(self):
        super().__init__()
        self.backend = DatabaseBackend()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('Cafeteria Database Interface')
        self.setGeometry(100, 100, 800, 600)

        self.stacked_widget = QStackedWidget()

        self.login_page = LoginPage(self.show_admin_view, self.show_student_view)
        self.admin_view = AdminView(self.show_login_page, self.backend)
        self.student_view = None  # Initialized dynamically

        self.stacked_widget.addWidget(self.login_page)
        self.stacked_widget.addWidget(self.admin_view)

        layout = QVBoxLayout()
        layout.addWidget(self.stacked_widget)
        self.setLayout(layout)

        self.show_login_page()

    def show_login_page(self):
        self.stacked_widget.setCurrentWidget(self.login_page)

    def show_admin_view(self):
        self.admin_view = AdminView(self.show_login_page, self.backend)
        self.stacked_widget.addWidget(self.admin_view)
        self.stacked_widget.setCurrentWidget(self.admin_view)
    
    def show_student_view(self, pid):
        self.student_view = StudentView(pid, self.show_login_page, self.backend)
        self.stacked_widget.addWidget(self.student_view)
        self.stacked_widget.setCurrentWidget(self.student_view)


class DatabaseBackend:
    def __init__(self):
        self.connection = pymysql.connect(
            host='localhost',
            user='root',
            password='P3nnyTh3D0g',
            database='cafeteria',
        )

        self.connection = pymysql.connect(
            host='localhost',
            user='rysch01',
            password='SQLP4ssword1!',
            database='cafeteria',
        )

        self.cursor = self.connection.cursor()

    def get_student_name(self, pid):
        query = "SELECT first_name, last_name FROM Students WHERE pid = %s"
        self.cursor.execute(query, (pid,))
        result = self.cursor.fetchone()
        return f"{result[0]} {result[1]}" if result else "Unknown"

    def get_account_balance(self, pid):
        query = "SELECT meal_balance FROM Students WHERE pid = %s"
        self.cursor.execute(query, (pid,))
        result = self.cursor.fetchone()
        return result[0] if result else 0.0

    def get_togo_boxes(self, pid):
        query = "SELECT to_go_boxes_remaining FROM Students WHERE pid = %s"
        self.cursor.execute(query, (pid,))
        result = self.cursor.fetchone()
        return result[0] if result else 0
    
    def get_menu_for_date(self, date, meal_type):
        query = """
        SELECT * FROM Menu WHERE available_date = %s AND meal_type = %s AND quantity > 0
        """
        self.cursor.execute(query, (date, meal_type))
        return self.cursor.fetchall()

    def process_transaction(self, pid, item_id, transaction_type, amount):
        self.cursor.execute("SELECT meal_balance, to_go_boxes FROM Students WHERE pid = %s", (pid,))
        student = self.cursor.fetchone()
        if not student:
            raise Exception("Student not found.")
        balance, to_go_boxes = student

        if transaction_type == "meal swipe" and balance < amount:
            raise Exception("Insufficient balance.")
        if transaction_type == "to-go box checkout" and to_go_boxes >= 3:
            raise Exception("To-go box limit exceeded.")

        # Update balance and boxes
        if transaction_type == "meal swipe":
            self.cursor.execute("UPDATE Students SET meal_balance = meal_balance - %s WHERE pid = %s", (amount, pid))
        elif transaction_type == "to-go box checkout":
            self.cursor.execute("UPDATE Students SET to_go_boxes = to_go_boxes + 1 WHERE pid = %s", (pid,))

        # Update menu quantity
        self.cursor.execute("UPDATE Menu SET quantity = quantity - 1 WHERE item_id = %s", (item_id,))

        # Insert into Transactions and DiningHistory
        self.cursor.execute(
            "INSERT INTO Transactions (student_id, item_id, transaction_type, transaction_date, amount) VALUES (%s, %s, %s, CURDATE(), %s)",
            (pid, item_id, transaction_type, amount)
        )
        self.cursor.execute(
            "INSERT INTO DiningHistory (student_id, transaction_date, transaction_type, item_id, meal_type) VALUES (%s, CURDATE(), %s, %s, %s)",
            (pid, transaction_type, item_id, "breakfast")  # Adjust meal_type dynamically if needed
        )
        self.db.commit()

    def get_all_students(self):
        query = "SELECT pid, CONCAT(first_name, ' ', last_name), meal_balance, to_go_boxes FROM Students"
        self.cursor.execute(query)
        return self.cursor.fetchall()

    def get_all_menu_items(self):
        query = "SELECT item_id, item_name, quantity, price, meal_type FROM Menu"
        self.cursor.execute(query)
        return self.cursor.fetchall()

    def get_all_transactions(self):
        query = """
        SELECT transaction_id, student_id, item_id, transaction_type, amount
        FROM Transactions
        """
        self.cursor.execute(query)
        return self.cursor.fetchall()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()
    sys.exit(app.exec_())


: 

In [ ]:
### Be sure to close at the end
### Note that if you close and then try to perform new queries you'll get an InterfaceError: (0, '') error and need to reopen conneciton
db.close()

NameError: name 'db' is not defined